In [2]:
from setup import general
from setup import stat
from setup import r

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import pingouin as pg
from pingouin import ancova # anconva
from pingouin import multivariate_normality # for pearson correlation
from rpy2 import robjects

In [1]:
%store -r Scale_pt_2 Scale_hc nreps Con Pt_included

# Analysis
患者后测与健康人行为评分的均值比较

In [3]:
hc_pt_2 =[]
scales = Scale_pt_2.columns[1:]
for scale in scales:
    lev = stats.levene(Scale_hc[scale],Scale_pt_2[scale])
    equal_var = True if lev[1]>0.05 else False   
    t_result = stats.ttest_ind(Scale_hc[scale],Scale_pt_2[scale], equal_var=equal_var)

    perm_ind_t_p = robjects.r['perm_ind_t'](r.to_csv_r(Scale_hc, 'hc', scale), r.to_csv_r(Scale_pt_2, 'pt', scale), 3 + len(scale), nreps=nreps)

    # set age as covariance
    ancova_a = Scale_hc[[scale]];    ancova_a.loc[:,'Type'] = [1 for i in range(ancova_a.shape[0])]
    ancova_b = Scale_pt_2[[scale]];    ancova_b.loc[:,'Type'] = [2 for i in range(ancova_b.shape[0])]
    ancova_df = pd.concat([ancova_a,ancova_b])
    ancova_age_1 = general.get_control_info(Con,ancova_df, 'Age'); ancova_age_2 = general.get_patient_info(Pt_included,ancova_df, 'Age')
    ancova_age = pd.DataFrame(pd.concat([ancova_age_1,ancova_age_2]))
    ancova_df = ancova_df.join(ancova_age)
    anc = ancova(data=ancova_df, dv=str(scale), between='Type', covar='Age')
    
    res = list(stats.shapiro(Scale_hc[scale])+stats.shapiro(Scale_pt_2[scale])+lev+tuple([round(t_result[0],5),round(t_result[1],5)])+tuple(perm_ind_t_p))
    res.append(anc.loc[0,'p-unc'])
    hc_pt_2.append(res)
hc_pt_df_2 = pd.DataFrame(hc_pt_2, index=scales, columns=['hc_norm_stat','hc_norm_p', 'pt_norm_stat','pt_norm_p','levene_stat', 'levene_p','ind_t','ind_t_p','perm_ind_t_p','ancova_p'])


/home/jzni/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/jzni/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/jzni/anaconda3/lib/python3.8/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input dat

,hc_norm_stat,hc_norm_p,pt_norm_stat,pt_norm_p,levene_stat,levene_p,ind_t,ind_t_p,perm_ind_t_p,ancova_p
VAS,1.000000,1.000000,0.869040,0.032665,12.516213,1.294652e-03,-5.11046,0.00016,0.0000,4.641114e-06
SFMPQ,1.000000,1.000000,0.906621,0.120181,41.415625,3.576198e-07,-4.35765,0.00066,0.0000,5.468185e-05
PDQ,1.000000,1.000000,0.907326,0.123211,19.006486,1.332321e-04,-5.77255,0.00005,0.0000,7.251347e-07
PSQ,0.961290,0.626715,0.937511,0.352098,1.102034,3.019355e-01,2.26223,0.03084,0.0292,3.179522e-02
FPQ_Severe,0.890967,0.040095,0.954390,0.596020,0.508985,4.809153e-01,1.35615,0.18485,0.1876,2.171618e-01
FPQ_Mild,0.935949,0.246736,0.902823,0.105117,0.014678,9.043523e-01,1.11117,0.27504,0.2706,3.000647e-01
FPQ_Medical,0.966361,0.727152,0.918924,0.185486,0.175462,6.781904e-01,1.66093,0.10681,0.1074,1.226846e-01
FPQ_Total,0.955960,0.525896,0.950469,0.531861,0.098584,7.556389e-01,1.66317,0.10636,0.1028,1.244248e-01
PCS_1,0.941694,0.309709,0.957838,0.654884,1.715920,1.998436e-01,-1.20772,0.23629,0.2304,2.667321e-01
PCS_2,0.867080,0.015932,0.888287,0.063180,1.015528,3.213838e-01,-1.09562,0.28168,0.2750,2.490446e-01


# Results

In [4]:
hc_pt_df_2

,hc_norm_stat,hc_norm_p,pt_norm_stat,pt_norm_p,levene_stat,levene_p,ind_t,ind_t_p,perm_ind_t_p,ancova_p
VAS,1.000000,1.000000,0.869040,0.032665,12.516213,1.294652e-03,-5.11046,0.00016,0.0000,4.641114e-06
SFMPQ,1.000000,1.000000,0.906621,0.120181,41.415625,3.576198e-07,-4.35765,0.00066,0.0000,5.468185e-05
PDQ,1.000000,1.000000,0.907326,0.123211,19.006486,1.332321e-04,-5.77255,0.00005,0.0000,7.251347e-07
PSQ,0.961290,0.626715,0.937511,0.352098,1.102034,3.019355e-01,2.26223,0.03084,0.0292,3.179522e-02
FPQ_Severe,0.890967,0.040095,0.954390,0.596020,0.508985,4.809153e-01,1.35615,0.18485,0.1876,2.171618e-01
FPQ_Mild,0.935949,0.246736,0.902823,0.105117,0.014678,9.043523e-01,1.11117,0.27504,0.2706,3.000647e-01
FPQ_Medical,0.966361,0.727152,0.918924,0.185486,0.175462,6.781904e-01,1.66093,0.10681,0.1074,1.226846e-01
FPQ_Total,0.955960,0.525896,0.950469,0.531861,0.098584,7.556389e-01,1.66317,0.10636,0.1028,1.244248e-01
PCS_1,0.941694,0.309709,0.957838,0.654884,1.715920,1.998436e-01,-1.20772,0.23629,0.2304,2.667321e-01
PCS_2,0.867080,0.015932,0.888287,0.063180,1.015528,3.213838e-01,-1.09562,0.28168,0.2750,2.490446e-01


In [5]:
hc_pt_result_2 = general.create_final_table(hc_pt_df_2, Scale_hc, Scale_pt_2, columns=['healthy control', 'patient post test'])
%store hc_pt_result_2
hc_pt_result_2 

Stored 'hc_pt_result_2' (DataFrame)


,healthy control,patient post test,t value,p value,ancova p value
VAS,0.0 ± 0.0,23.89 ± 18.1,-5.11,0.00000,4.641114e-06
SFMPQ,0.0 ± 0.0,32.07 ± 28.5,-4.36,0.00066,5.468185e-05
PDQ,0.0 ± 0.0,8.93 ± 5.99,-5.77,0.00005,7.251347e-07
PSQ,74.11 ± 17.18,57.46 ± 24.96,2.26,0.03084,3.179522e-02
FPQ_Severe,36.83 ± 8.53,33.0 ± 7.51,1.36,0.18760,2.171618e-01
FPQ_Mild,23.56 ± 7.71,20.54 ± 7.83,1.11,0.27504,3.000647e-01
FPQ_Medical,27.72 ± 9.71,22.15 ± 9.45,1.66,0.10681,1.226846e-01
FPQ_Total,88.17 ± 23.43,74.92 ± 21.97,1.66,0.10636,1.244248e-01
PCS_1,5.5 ± 3.42,7.21 ± 4.72,-1.21,0.23629,2.667321e-01
PCS_2,3.22 ± 3.37,4.64 ± 4.08,-1.10,0.27500,2.490446e-01




**显著的评分**
* VAS
* SFMPQ
* PDQ
* SAI
* HAMA
* BDI (协方差分析边缘显著)       

**边缘显著评分**
* HAMD (置换检验\协方差结果边缘显著）
* PSQI
